# Mail Spam detection : Machine Learning

In [57]:
import numpy as np   
import pandas as pd  
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# 1) LOADING DATA

In [58]:
dt = pd.read_csv("spam.csv")  
dt.head(10)

,type,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [59]:
dt['spam'] = dt['type'].map( {'spam': 1, 'ham': 0} ).astype(int)
dt.head(5)

,type,text,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [60]:
print("COLUMS IN THE GIVEN DATA:")
for col in dt.columns: 
    print(col) 

COLUMS IN THE GIVEN DATA:
type
text
spam


In [61]:
t=len(dt['type'])
print("NO OF ROWS IN REVIEW COLUMN:",t)
t=len(dt['text'])
print("NO OF ROWS IN liked COLUMN:",t)

NO OF ROWS IN REVIEW COLUMN: 116
NO OF ROWS IN liked COLUMN: 116


# 2)Tokenization

In [62]:
dt['text'][3]#before   

'U dun say so early hor... U c already then say...'

In [63]:
def tokenizer(text):
    return text.split()

In [64]:
dt['text']=dt['text'].apply(tokenizer)

In [65]:
dt['text'][3]#after 

['U',
 'dun',
 'say',
 'so',
 'early',
 'hor...',
 'U',
 'c',
 'already',
 'then',
 'say...']

# 3) STEMMING 

In [66]:
dt['text'][3]#before 

['U',
 'dun',
 'say',
 'so',
 'early',
 'hor...',
 'U',
 'c',
 'already',
 'then',
 'say...']

In [67]:
from nltk.stem.snowball import SnowballStemmer
porter = SnowballStemmer("english", ignore_stopwords=False)

In [68]:
def stem_it(text):
    return [porter.stem(word) for word in text]

In [69]:
dt['text']=dt['text'].apply(stem_it)

In [70]:
dt['text'][] #after stemming 

['u',
 'dun',
 'say',
 'so',
 'earli',
 'hor...',
 'u',
 'c',
 'alreadi',
 'then',
 'say...']

# 4) LEMMITIZATION 

In [71]:
dt['text'][80] #before

['sorry,', "i'll", 'call', 'later']

In [72]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 

In [73]:
def lemmit_it(text):
    return [lemmatizer.lemmatize(word, pos ="a") for word in text]

In [74]:
dt['text']=dt['text'].apply(lemmit_it)

In [75]:
dt['text'][80] #before

['sorry,', "i'll", 'call', 'late']

# 5)STOPWORD REMOVAL

In [76]:
dt['text'][111] #before 

['what', 'is', 'the', 'plural', 'of', 'the', 'noun', 'research?']

In [77]:
#from nltk.corpus import stopwords
#print(stopwords.words('english'))
#set(['a','t','d','y'])

In [78]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [79]:
def stop_it(text):
    review = [word for word in text if not word in stop_words ] 
    return review

In [80]:
dt['text']=dt['text'].apply(stop_it)

In [81]:
dt['text'][111] #after 

['plural', 'noun', 'research?']

In [82]:
dt.head(10)

,type,text,spam
0,ham,"[go, jurong, point,, crazy.., avail, onli, bug...",0
1,ham,"[ok, lar..., joke, wif, u, oni...]",0
2,spam,"[free, entri, 2, wkli, comp, win, fa, cup, fin...",1
3,ham,"[u, dun, say, earli, hor..., u, c, alreadi, sa...",0
4,ham,"[nah, think, goe, usf,, live, around, though]",0
5,spam,"[freemsg, hey, darl, 3, week, word, back!, i'd...",1
6,ham,"[even, brother, like, speak, me., treat, like,...",0
7,ham,"[per, request, mell, mell, (oru, minnaminungin...",0
8,spam,"[winner!!, valu, network, custom, select, rece...",1
9,spam,"[mobil, 11, month, more?, u, r, entitl, updat,...",1


In [83]:
dt['text']=dt['text'].apply(' '.join)

In [84]:
dt.head()

,type,text,spam
0,ham,"go jurong point, crazy.. avail onli bugi n gre...",0
1,ham,ok lar... joke wif u oni...,0
2,spam,free entri 2 wkli comp win fa cup final tkts 2...,1
3,ham,u dun say earli hor... u c alreadi say...,0
4,ham,"nah think goe usf, live around though",0


#  6) Transform Text Data into TDF /TF-IDF Vectors   

In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
y=dt.spam.values
x=tfidf.fit_transform(dt['text'])

In [86]:
from sklearn.model_selection import train_test_split
x_train,x_text,y_train,y_text=train_test_split(x,y,random_state=1,test_size=0.2,shuffle=False)

#  7) Classification using Logistic Regression

In [87]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression()
clf.fit(x_train,y_train)
y_pred=clf.predict(x_text)
from sklearn.metrics import accuracy_score
acc_log = accuracy_score(y_pred, y_text)*100
print("accuracy:",acc_log )

accuracy: 87.5


#   8) Classification using LinearSVC Accuracy 

In [88]:
from sklearn.svm import LinearSVC

linear_svc = LinearSVC(random_state=0)
linear_svc.fit(x_train, y_train)
y_pred = linear_svc.predict(x_text)
acc_linear_svc =accuracy_score(y_pred, y_text) * 100
print("accuracy:",acc_linear_svc)

accuracy: 87.5


###                        Saurabh Sharma